In [2]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd
import sys
from CCLE_postp_function import *
sys.path.insert(0, '../JKBio/')
import datanalytics as da 
import TerraFunction as terra
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
tc = TaigaClient()
from wand.image import Image as WImage
import numpy as np
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier
from bokeh.plotting import *
from bokeh.models import HoverTool
output_notebook()
from collections import OrderedDict

Loading BokehJS ...

In [3]:
namespace="broad-genomics-delivery"
workspace="Getz_IBM_CellLines_RNASeqData"
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_hg38_RNAseq"
source="ibm"
samplesetname="19Q3interim"
release = samplesetname

In [4]:
wm = dm.WorkspaceManager(namespace, workspace)
refwm = dm.WorkspaceManager(refnamespace, refworkspace)

# Generate sample set from new samples

In [12]:
CreateDatasetWithNewCellLines(wm,refwm)

,CCLE_name,ExonUsage_OutFN_hg38,ExonUsage_OutRobjFN_hg38,RNAseq_bai,RNAseq_bam,bam_file,bam_index,exon_usage_hg38_outFN,exon_usage_hg38_outRobjFN,fastq1,...,star_chimeric_bam_index,star_chimeric_junctions,star_junctions,star_junctions_pass1,star_logs,star_md_bam_file,star_md_bam_index,star_read_counts,star_transcriptome_bam,transcriptome_bam
sample_id,,,,,,,,,,,,,,,,,,,,,
dm_ACH-000001,NIHOVCAR3_OVARY,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-c7d47ebf-26a0-404c-8ef6-a1eb59c32e54/G...,gs://fc-c7d47ebf-26a0-404c-8ef6-a1eb59c32e54/G...,NaN,NaN,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,[gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd500...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,NaN
dm_ACH-000002,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-c7d47ebf-26a0-404c-8ef6-a1eb59c32e54/G...,gs://fc-c7d47ebf-26a0-404c-8ef6-a1eb59c32e54/G...,NaN,NaN,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,[gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd500...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,NaN
dm_ACH-000003,CACO2_LARGE_INTESTINE,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-effb3f55-962a-4b1f-b41d-63234d7e5735/s...,gs://fc-effb3f55-962a-4b1f-b41d-63234d7e5735/s...,NaN,NaN,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,[gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd500...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,NaN
dm_ACH-000006,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-c7d47ebf-26a0-404c-8ef6-a1eb59c32e54/G...,gs://fc-c7d47ebf-26a0-404c-8ef6-a1eb59c32e54/G...,NaN,NaN,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,[gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd500...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,NaN
dm_ACH-000007,LS513_LARGE_INTESTINE,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-c7d47ebf-26a0-404c-8ef6-a1eb59c32e54/G.

# run the pipeline

In [28]:
genes_count = pd.read_csv('temp/' + mainres['rsem_genes_expected_count'].split('/')[-1], sep='\t', compression='gzip')

In [29]:
genes_count

,Unnamed: 0,Unnamed: 0.1,gene_id,transcript_id(s),dm_ACH-000001,dm_ACH-000002,dm_ACH-000003,dm_ACH-000006,dm_ACH-000007,dm_ACH-000009,...,ibm_ACH-001997,ibm_ACH-001999,ibm_ACH-002001,ibm_ACH-002002,ibm_ACH-002003,ibm_ACH-002004,ibm_ACH-002005,ibm_ACH-002011,ibm_ACH-002015,ACH-001616
0,0,0,ENSG00000000003.14,"ENST00000373020.8,ENST00000494424.1,ENST000004...",4251.00,14.00,7568.00,13.00,1943.00,8900.00,...,41694.0,19180.0,1135.00,1103.00,895.00,4331.00,2037.00,39.00,21194.0,0.00
1,1,1,ENSG00000000005.5,"ENST00000373031.4,ENST00000485971.1",0.00,0.00,1.00,0.00,0.00,4.00,...,0.0,0.0,0.00,4.00,0.00,0.00,0.00,0.00,1.0,0.00
2,2,2,ENSG00000000419.12,"ENST00000371582.8,ENST00000371584.8,ENST000003...",5339.00,2843.00,10643.00,3098.00,3024.00,10472.00,...,66739.0,31970.0,5787.00,3330.00,6263.00,2688.00,5400.00,2895.00,27198.0,2598.00
3,3,3,ENSG00000000457.13,"ENST00000367770.5,ENST00000367771.10,ENST00000...",573.96,496.40,642.65,1125.90,1920.50,751.24,...,7319.4,7203.4,1000.60,679.60,1235.80,628.57,665.09,862.62,4568.8,883.56
4,4,4,ENSG00000000460.16,"ENST00000286031.10,ENST00000359326.8,ENST00000...",1398.00,1272.60,1367.30,1960.10,1441.50,1796.80,...,11022.0,2734.6,1689.40,897.40,951.19,523.43,966.91,1731.40,5527.2,281.44
5,5,5,ENSG00000000938.12,"ENST00000374003.7,ENST00000374004.5,ENST000003...",1.00,2444.00,0.00,1429.00,1.00,0.00,...,5.0,78.0,0.00,7.00,1.00,1.00,5.00,0.00,12.0,37.00
6,6,6,ENSG00000000971.15,"ENST00000359637.2,ENST00000367429.8,ENST000004...",72.00,29.00,8.00,2067.00,3.00,4.00,...,16110.0,25470.0,19.17,2256.70,37.00,137.47,0.00,423.68,20130.0,0.00
7,7,7,ENSG00000001036.13,"ENST00000002165.10,ENST00000367585.1,ENST00000...",4020.40,2441.50,11743.00,3918.30,6014.50,14061.00,...,91361.0,29799.0,1538.90,2140.90,2198.90,9847.80,2292.90,5704.40,15214.0,180.00
8,8,8,ENSG00000001084.12,"ENST00000229416.10,ENST00000504353.1,ENST00000...",2365.10,3667.00,3833.00,3466.00,4723.00,6104.00,...,22693.0,16030.0,1835.00,1549.00,2337.00,2024.00,2389.00,4574.00,7974.0,5720.00
9,9,9,ENSG00000001167.14,"ENST00000341376.10,ENST00000353205.5",1920.00,1493.00,2939.00,5040.00,2708.00,3593.00,...,13383.0,15955.0,1576.00,1621.00,1963.00,2775.00,1683.00,7958.00,13264.0,1986.00


In [24]:
samtofastq = refwm.get_config("samtofastq_v1-0_BETA_cfg")
samtofastq

{'deleted': False,
 'inputs': {'samtofastq_workflow.samtofastq.prefix': 'this.name',
  'samtofastq_workflow.samtofastq.disk_space': '50',
  'samtofastq_workflow.samtofastq.num_threads': '1',
  'samtofastq_workflow.samtofastq.memory': '3',
  'samtofastq_workflow.samtofastq.num_preempt': '5',
  'samtofastq_workflow.samtofastq.input_bam_cram': 'this.RNAseq_bam'},
 'methodConfigVersion': 1,
 'methodRepoMethod': {'methodName': 'samtofastq_v1-0_BETA',
  'methodVersion': 5,
  'methodNamespace': 'broadinstitute_gtex',
  'methodUri': 'agora://broadinstitute_gtex/samtofastq_v1-0_BETA/5',
  'sourceRepo': 'agora'},
 'name': 'samtofastq_v1-0_BETA_cfg',
 'namespace': 'broadinstitute_gtex',
 'outputs': {'samtofastq_workflow.samtofastq.fastq1': 'this.fastq1',
  'samtofastq_workflow.samtofastq.fastq2': 'this.fastq2'},
 'prerequisites': {},
 'rootEntityType': 'sample'}

In [25]:
samtofastq['samtofastq_workflow.samtofastq.input_bam_cram']= 'this.WES_bam'
refwm.update_config(samtofastq)
submission_id = refwm.create_submission(samtofastq['name'], samplesetname,'sample_set',expression='this.samples')


Successfully created submission b09e54b1-4280-440a-9dd3-6926ca793056.


'b09e54b1-4280-440a-9dd3-6926ca793056'

In [ ]:
wait_for_submission(refwm, submission_id1)

In [26]:
star = refwm.get_config("star_v1-0_BETA_cfg")
star

{'deleted': False,
 'inputs': {'star_workflow.star.outSAMattrRGline': '"ID:rg1 SM:sm1"',
  'star_workflow.star.num_threads': '8',
  'star_workflow.star.alignSoftClipAtReferenceEnds': '"Yes"',
  'star_workflow.star.alignMatesGapMax': '1000000',
  'star_workflow.star.outFilterType': '"BySJout"',
  'star_workflow.star.prefix': 'this.name',
  'star_workflow.star.num_preempt': '5',
  'star_workflow.star.alignSJoverhangMin': '8',
  'star_workflow.star.chimOutJunctionFormat': '1',
  'star_workflow.star.memory': '52',
  'star_workflow.star.fastq2': 'this.fastq2',
  'star_workflow.star.outFilterMismatchNoverLmax': '0.1',
  'star_workflow.star.alignIntronMax': '1000000',
  'star_workflow.star.star_index': 'workspace.star_index',
  'star_workflow.star.outSAMattributes': '"NH HI AS nM NM ch"',
  'star_workflow.star.outSAMstrandField': '"intronMotif"',
  'star_workflow.star.limitSjdbInsertNsj': '1200000',
  'star_workflow.star.chimSegmentMin': '15',
  'star_workflow.star.chimJunctionOverhangMin': '

In [29]:
refwm.update_config(star)
submission_id = refwm.create_submission(star['name'], samplesetname,'sample_set',expression='this.samples')


Successfully updated configuration broadinstitute_gtex/star_v1-0_BETA_cfg
Successfully created submission f60e09f0-98cd-4982-930b-aa2896f6dbde.


'f60e09f0-98cd-4982-930b-aa2896f6dbde'

In [ ]:
wait_for_submission(refwm, submission_id1)

In [30]:
rsem = refwm.get_config("rsem_v1-0_BETA_cfg")
rsem

{'deleted': False,
 'inputs': {'rsem_workflow.rsem.transcriptome_bam': 'this.star_transcriptome_bam',
  'rsem_workflow.rsem.max_frag_len': '1000',
  'rsem_workflow.rsem.memory': '50',
  'rsem_workflow.rsem.prefix': 'this.name',
  'rsem_workflow.rsem.disk_space': '100',
  'rsem_workflow.rsem.estimate_rspd': '"true"',
  'rsem_workflow.rsem.is_stranded': '"false"',
  'rsem_workflow.rsem.rsem_reference': 'workspace.rsem_reference',
  'rsem_workflow.rsem.num_preempt': '5',
  'rsem_workflow.rsem.num_threads': '2'},
 'methodConfigVersion': 2,
 'methodRepoMethod': {'methodName': 'rsem_v1-0_BETA',
  'methodVersion': 4,
  'methodNamespace': 'broadinstitute_gtex',
  'methodUri': 'agora://broadinstitute_gtex/rsem_v1-0_BETA/4',
  'sourceRepo': 'agora'},
 'name': 'rsem_v1-0_BETA_cfg',
 'namespace': 'broadinstitute_gtex',
 'outputs': {'rsem_workflow.rsem.genes': 'this.rsem_genes',
  'rsem_workflow.rsem.isoforms': 'this.rsem_isoforms'},
 'prerequisites': {},
 'rootEntityType': 'sample'}

In [31]:
refwm.update_config(rsem)
submission_id1 = refwm.create_submission(rsem['name'], samplesetname,'sample_set',expression='this.samples')

Successfully updated configuration broadinstitute_gtex/rsem_v1-0_BETA_cfg
Successfully created submission 25b86d44-d5d3-481a-9c29-5438c9905f44.


In [ ]:
wait_for_submission(refwm, submission_id1)

In [32]:
fusion = refwm.get_config("hg38_STAR_fusion")
fusion

{'deleted': False,
 'inputs': {'trinity_cleaned.StarFusion.ref_genome_fa_star_idx_files': 'workspace.hg38_star_fusion_ctat_star_index_files',
  'trinity_cleaned.samtofastq.num_preempt': '4',
  'trinity_cleaned.StarFusion.memory': '52',
  'trinity_cleaned.remove_intermediate_files': 'true',
  'trinity_cleaned.StarFusion.num_preempt': '2',
  'trinity_cleaned.StarFusion.ctat_genome_lib_build_dir_files': 'workspace.hg38_star_fusion_ctat_files',
  'trinity_cleaned.prefix': 'this.name',
  'trinity_cleaned.StarFusion.num_threads': '8',
  'trinity_cleaned.samtofastq.memory': '3',
  'trinity_cleaned.samtofastq.num_threads': '1',
  'trinity_cleaned.samtofastq.input_bam': 'this.RNAseq_bam',
  'trinity_cleaned.StarFusion.disk_space': '250',
  'trinity_cleaned.StarFusion.docker': '"trinityctat/ctatfusion:1.6.0"',
  'trinity_cleaned.samtofastq.disk_space': '50'},
 'methodConfigVersion': 4,
 'methodRepoMethod': {'methodName': 'STAR_fusion',
  'methodVersion': 14,
  'methodNamespace': 'gkugener',
  'm

In [45]:
fusion['inputs']['trinity_cleaned.samtofastq.input_bam'] = 'this.WES_bam'
refwm.update_config(fusion)
submission_id2 = refwm.create_submission(fusion['name'], samplesetname,'sample_set',expression='this.samples')

Successfully updated configuration gkugener/hg38_STAR_fusion
Successfully created submission ee4fd082-407b-4255-ae24-8734e626d890.


In [ ]:
wait_for_submission(refwm, submission_id1)

In [28]:
aggregate = refwm.get_config("rsem_aggregate_results_v1-0_BETA_cfg")
aggregate

{'deleted': False,
 'inputs': {'rsem_aggregate_results_workflow.rsem_aggregate_results.rsem_genes': 'this.samples.rsem_genes',
  'rsem_aggregate_results_workflow.rsem_aggregate_results.num_preempt': '5',
  'rsem_aggregate_results_workflow.rsem_aggregate_results.prefix': 'this.name',
  'rsem_aggregate_results_workflow.rsem_aggregate_results.num_threads': '2',
  'rsem_aggregate_results_workflow.rsem_aggregate_results.rsem_isoforms': 'this.samples.rsem_isoforms',
  'rsem_aggregate_results_workflow.rsem_aggregate_results.disk_space': '50',
  'rsem_aggregate_results_workflow.rsem_aggregate_results.memory': '13'},
 'methodConfigVersion': 3,
 'methodRepoMethod': {'methodName': 'rsem_aggregate_results_v1-0_BETA',
  'methodVersion': 3,
  'methodNamespace': 'broadinstitute_gtex',
  'methodUri': 'agora://broadinstitute_gtex/rsem_aggregate_results_v1-0_BETA/3',
  'sourceRepo': 'agora'},
 'name': 'rsem_aggregate_results_v1-0_BETA_cfg',
 'namespace': 'broadinstitute_gtex',
 'outputs': {'rsem_aggrega

In [50]:
updateAllSampleSet('CCLE_'+samplesetname, Allsample_setname='DM19Q2_PATHS_CORRECTED_V2')

In [57]:
submission_id1 = refwm.create_submission(aggregate['name'], 'All_samples')

Successfully created submission 2840c346-6310-4bbb-ad3b-053f6692d4d7.


## Expression post processing

In [58]:
wait_for_submission(refwm, submission_id1)

NameError: name 'wait_for_submission' is not defined

In [4]:
%%R
release <- '19Q3interim'

In [5]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')

2019-06-20 06:06:20::WARNING  R[write to console]: 
Attaching package: ‘dplyr’


2019-06-20 06:06:20::WARNING  R[write to console]: The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


2019-06-20 06:06:20::WARNING  R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


2019-06-20 06:06:20::WARNING  R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


2019-06-20 06:06:24::WARNING  R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

2019-06-20 06:06:24::WARNING  R[write to console]: ✔ ggplot2 3.1.1     ✔ readr   1.3.1
✔ tibble  2.1.3     ✔ purrr   0.3.2
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ ggplot2 3.1.1     ✔ forcats 0.4.0

2019-06-20 06:06:24::WARNING  R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts()

2019-06-20 06:06:30::WARNING  R[write to console]: The following objects are masked from ‘package:data.table’:

    first, second


2019-06-20 06:06:30::WARNING  R[write to console]: The following object is masked from ‘package:plotly’:

    rename


2019-06-20 06:06:30::WARNING  R[write to console]: The following object is masked from ‘package:tidyr’:

    expand


2019-06-20 06:06:30::WARNING  R[write to console]: The following objects are masked from ‘package:dplyr’:

    first, rename


2019-06-20 06:06:30::WARNING  R[write to console]: The following object is masked from ‘package:plyr’:

    rename


2019-06-20 06:06:30::WARNING  R[write to console]: The following object is masked from ‘package:base’:

    expand.grid


2019-06-20 06:06:30::WARNING  R[write to console]: Loading required package: IRanges

2019-06-20 06:06:31::WARNING  R[write to console]: 
Attaching package: ‘IRanges’


2019-06-20 06:06:31::WARNING  R[write to console]: The following object is masked from ‘package:

Fetching https://cds.team/taiga/api/dataset/msigdb-gene-set-collections-8453/2 
Status 200 


2019-06-20 06:06:38::WARNING  R[write to console]: No encoding supplied: defaulting to UTF-8.



loading cached data version from  ~/.taiga/msigdb-gene-set-collections-8453_2.toc 
Checking for cached file ~/.taiga/495ea37b8d744a428946a5628c5fe7ce_gsc-data.raw 


In [110]:
%%R
source('CCLE_postp_function.R')

In [14]:
res = refwm.get_sample_sets().loc['all_samples']
res

fusions_star                                                                     NaN
mergedExonUsageRObj_hg38                                                         NaN
rsem_genes_expected_count                                                        NaN
rsem_genes_tpm                                                                   NaN
rsem_transcripts_expected_count                                                  NaN
rsem_transcripts_isopct                                                          NaN
rsem_transcripts_tpm                                                             NaN
samples                            [dm_ACH-000001, dm_ACH-000002, dm_ACH-000003, ...
Name: all_samples, dtype: object

In [11]:
res = refwm.get_sample_sets().loc['All_samples']
rsem_genes_expected_count = res['rsem_genes_expected_count']
rsem_genes_tpm = res['rsem_genes_tpm']
rsem_transcripts_tpm = res['rsem_transcripts_tpm']
! gsutil cp $rsem_genes_expected_count "temp/" & gsutil cp $rsem_genes_tpm "temp/" & gsutil cp $rsem_transcripts_tpm "temp/"

CommandException: No URLs matched: nan
CommandException: No URLs matched: nan
CommandException: No URLs matched: nan


In [14]:
%%R
# alternative one
download_paths <- list(
  counts_genes='temp/DM19Q2_PATHS_CORRECTED_V2.rsem_genes_expected_count.txt.gz',
  tpm_genes='temp/DM19Q2_PATHS_CORRECTED_V2.rsem_genes_tpm.txt.gz',
  tpm_transcripts='temp/DM19Q2_PATHS_CORRECTED_V2.rsem_transcripts_tpm.txt.gz')

In [7]:
%%R
# These files are downloaded from FireCloud/Terra
download_paths <- list(
  tpm_genes='temp/expression.genes.tpm.txt.gz',
  tpm_transcripts='temp/expression.transcript.tpm.txt.gz',
  counts_genes='temp/expression.expectedcount.txt.gz')

UsageError: Cell magic `%%R` not found.


In [8]:
%%R
previous_release_tpm <- load.from.taiga(data.name='depmap-rnaseq-expression-data-363a',
  data.file='CCLE_depMap_19Q2_TPM') %>% 
  set_colnames(stringr::str_extract(string=colnames(.), pattern='ENSG[0-9]+'))

Fetching https://cds.team/taiga/api/datafile?format=metadata&dataset_permaname=depmap-rnaseq-expression-data-363a&datafile_name=CCLE_depMap_19Q2_TPM 
Status 200 


2019-06-20 06:06:41::WARNING  R[write to console]: No encoding supplied: defaulting to UTF-8.

2019-06-20 06:06:41::WARNING  R[write to console]: Loading from cached file /home/jeremie/.taiga/7c31f9e4f7554078af3e0726a8afaf44_ccle-depmap-19q2-tpm.rds



In [111]:
%%R
tpm_transcripts = readTranscripts(download_paths$tpm_transcripts)
counts_genes = readCounts(download_paths$counts_genes)
tpm_genes = readTPM(download_paths$tpm_genes)

2019-06-20 08:30:38::WARNING  R[write to console]: Parsed with column specification:
cols(
  .default = col_double(),
  transcript_id = col_character(),
  gene_id = col_character()
)

2019-06-20 08:30:38::WARNING  R[write to console]: See spec(...) for full column specifications.

2019-06-20 08:31:57::WARNING  R[write to console]: Parsed with column specification:
cols(
  .default = col_double(),
  gene_id = col_character(),
  `transcript_id(s)` = col_character()
)

2019-06-20 08:31:57::WARNING  R[write to console]: See spec(...) for full column specifications.

2019-06-20 08:32:33::WARNING  R[write to console]: Parsed with column specification:
cols(
  .default = col_double(),
  gene_id = col_character(),
  `transcript_id(s)` = col_character()
)

2019-06-20 08:32:33::WARNING  R[write to console]: See spec(...) for full column specifications.



In [112]:
%%R
# Fix the colnames (for cases where there are mixed values (CCLE_name and DepMap_IDs))
colnames(counts_genes) %<>% renameFunction(.)
colnames(tpm_genes) %<>% renameFunction(.)
colnames(tpm_transcripts) %<>% renameFunction(.)

### data exploration and QC

In [109]:
%%R
tpm_genes[,'ACH-001767']

# A tibble: 58,813 x 1
   `ACH-001767`
   <chr>       
 1 1.22        
 2 0.0         
 3 5.51        
 4 0.87        
 5 0.16        
 6 0.0         
 7 2.86        
 8 2.79        
 9 6.02        
10 1.04        
# … with 58,803 more rows


In [113]:
%%R 
# Quick QC
# We are looking for samples with a worrying amount of zeros
zero_threshold <- 39000
number_zeros <- apply(tpm_genes[,3:ncol(tpm_genes)], 2, FUN = function(x) length(x[x == 0]))
nzdf <- data.frame(CL=names(number_zeros), nz=number_zeros, stringsAsFactors = F)

In [114]:
%%R
number_zeros <- number_zeros[order(-number_zeros)]
number_zeros <- number_zeros[number_zeros < zero_threshold]
pass <- number_zeros %>% names()

# These samples failed
failed <- setdiff(colnames(tpm_genes), pass) %>% .[!(. %in% c('gene_id', 'transcript_id(s)'))]

counts_genes %<>% dplyr::select(c("gene_id","transcript_id(s)", pass))
tpm_genes %<>% dplyr::select(c("gene_id","transcript_id(s)", pass))
tpm_transcripts %<>% dplyr::select(c("transcript_id", "gene_id", pass))

In [115]:
%%R
failed

[1] "ACH-001767"


In [ ]:
%%R
# Plot of the samples that fail
plot <- ggplot(nzdf, aes(nz)) +
  geom_histogram(bins = 100, color='black', fill='white') +
  geom_vline(xintercept = zero_threshold, linetype=2) +
  geom_label_repel(data = nzdf %>% filter(nz > zero_threshold), aes(x=nz, y=0, label=CL), size=5, fill=rgb(1,1,1,0.5))

ggsave(plot, filename ='temp/ggplot.pdf', width=20, height = 20)

In [ ]:
img = WImage(filename='temp/ggplot.pdf')
img

In [117]:
%%R
write.table(
  counts_genes, 
  file = paste0('temp/expression.', release,'.counts.tsv'), 
  sep = '\t', row.names = F, quote = F)
write.table(
  tpm_genes, 
  file = paste0('temp/expression.', release,'.tpm.tsv'), 
  sep = '\t', row.names = F, quote = F)
write.table(
  tpm_transcripts, 
  file = paste0('temp/expression.', release,'.transcripts.tsv'),
  sep = '\t', row.names = F, quote = F)

# Validation

In [ ]:
counts_genes = pd.read_csv('temp/expression.'+ release + '.counts.tsv', sep='\t')
tpm_genes = pd.read_csv('temp/expression.'+ release + '.tpm.tsv', sep='\t')
tpm_transcripts = pd.read_csv('temp/expression.'+ release + '.transcripts.tsv', sep='\t')

In [67]:
tpm_genes

,gene_id,transcript_id(s),ACH-000001,ACH-000002,ACH-000003,ACH-000006,ACH-000007,ACH-000009,ACH-000010,ACH-000011,...,ACH-001997,ACH-001999,ACH-002001,ACH-002002,ACH-002003,ACH-002004,ACH-002005,ACH-002011,ACH-002015,ACH-001616
0,ENSG00000000003.14,"ENST00000373020.8,ENST00000494424.1,ENST000004...",4251.00,14.00,7568.00,13.00,1943.00,8900.00,2083.00,2634.00,...,41694.0,19180.0,1135.00,1103.00,895.00,4331.00,2037.00,39.00,21194.0,0.00
1,ENSG00000000005.5,"ENST00000373031.4,ENST00000485971.1",0.00,0.00,1.00,0.00,0.00,4.00,0.00,0.00,...,0.0,0.0,0.00,4.00,0.00,0.00,0.00,0.00,1.0,0.00
2,ENSG00000000419.12,"ENST00000371582.8,ENST00000371584.8,ENST000003...",5339.00,2843.00,10643.00,3098.00,3024.00,10472.00,6662.00,2333.00,...,66739.0,31970.0,5787.00,3330.00,6263.00,2688.00,5400.00,2895.00,27198.0,2598.00
3,ENSG00000000457.13,"ENST00000367770.5,ENST00000367771.10,ENST00000...",573.96,496.40,642.65,1125.90,1920.50,751.24,910.51,725.41,...,7319.4,7203.4,1000.60,679.60,1235.80,628.57,665.09,862.62,4568.8,883.56
4,ENSG00000000460.16,"ENST00000286031.10,ENST00000359326.8,ENST00000...",1398.00,1272.60,1367.30,1960.10,1441.50,1796.80,2494.50,479.59,...,11022.0,2734.6,1689.40,897.40,951.19,523.43,966.91,1731.40,5527.2,281.44
5,ENSG00000000938.12,"ENST00000374003.7,ENST00000374004.5,ENST000003...",1.00,2444.00,0.00,1429.00,1.00,0.00,2.00,0.00,...,5.0,78.0,0.00,7.00,1.00,1.00,5.00,0.00,12.0,37.00
6,ENSG00000000971.15,"ENST00000359637.2,ENST00000367429.8,ENST000004...",72.00,29.00,8.00,2067.00,3.00,4.00,1421.80,57.84,...,16110.0,25470.0,19.17,2256.70,37.00,137.47,0.00,423.68,20130.0,0.00
7,ENSG00000001036.13,"ENST00000002165.10,ENST00000367585.1,ENST00000...",4020.40,2441.50,11743.00,3918.30,6014.50,14061.00,11666.00,5544.20,...,91361.0,29799.0,1538.90,2140.90,2198.90,9847.80,2292.90,5704.40,15214.0,180.00
8,ENSG00000001084.12,"ENST00000229416.10,ENST00000504353.1,ENST00000...",2365.10,3667.00,3833.00,3466.00,4723.00,6104.00,2376.00,2809.00,...,22693.0,16030.0,1835.00,1549.00,2337.00,2024.00,2389.00,4574.00,7974.0,5720.00
9,ENSG00000001167.14,"ENST00000341376.10,ENST00000353205.5",1920.00,1493.00,2939.00,5040.00,2708.00,3593.00,3539.00,1392.00,...,13383.0,15955.0,1576.00,1621.00,1963.00,2775.00,1683.00,7958.00,13264.0,1986.00


In [5]:
prev = tc.get(name='depmap-expression-87f8', version=7, file='DM19Q2.tpm')

In [72]:
prev

,gene_id,transcript_id(s),ACH-001097,ACH-001485,ACH-001396,ACH-001804,ACH-000534,ACH-000742,ACH-000836,ACH-000545,...,ACH-000305,ACH-000603,ACH-000916,ACH-000296,ACH-000978,ACH-000904,ACH-000110,ACH-000261,ACH-000031,ACH-000682
0,ENSG00000000003.14,"ENST00000373020.8,ENST00000494424.1,ENST000004...",0.00,0.00,6.38,29.59,0.79,12.20,23.80,18.05,...,16.78,0.88,30.95,40.71,31.05,18.93,36.46,20.82,25.26,61.96
1,ENSG00000000005.5,"ENST00000373031.4,ENST00000485971.1",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.11,0.03,0.02,0.05,0.02,0.02,0.09
2,ENSG00000000419.12,"ENST00000371582.8,ENST00000371584.8,ENST000003...",24.41,53.03,21.19,98.42,40.53,64.33,103.27,49.25,...,64.09,52.45,58.65,92.81,91.01,62.60,107.71,106.55,106.33,62.27
3,ENSG00000000457.13,"ENST00000367770.5,ENST00000367771.10,ENST00000...",2.39,4.19,4.09,2.81,5.44,3.57,5.93,9.86,...,4.14,4.68,13.07,5.03,5.12,8.10,10.61,5.28,5.85,6.67
4,ENSG00000000460.16,"ENST00000286031.10,ENST00000359326.8,ENST00000...",10.75,5.21,9.50,4.67,20.71,4.79,15.91,8.68,...,2.16,7.95,9.90,19.43,8.47,27.77,21.49,23.38,11.31,6.56
5,ENSG00000000938.12,"ENST00000374003.7,ENST00000374004.5,ENST000003...",19.52,17.81,0.41,0.00,29.66,0.11,0.07,1.91,...,0.02,0.02,0.03,0.03,0.01,0.22,0.11,0.03,0.04,0.06
6,ENSG00000000971.15,"ENST00000359637.2,ENST00000367429.8,ENST000004...",0.56,0.46,0.00,28.52,0.45,60.13,1.22,100.09,...,34.62,0.05,24.58,0.07,0.05,0.08,1.22,0.61,7.55,114.31
7,ENSG00000001036.13,"ENST00000002165.10,ENST00000367585.1,ENST00000...",3.60,11.22,25.57,108.33,11.00,124.66,52.06,88.18,...,89.91,39.18,10.91,84.68,40.42,26.84,45.44,83.47,121.45,52.54
8,ENSG00000001084.12,"ENST00000229416.10,ENST00000504353.1,ENST00000...",2.84,7.39,10.32,13.82,6.65,35.56,26.90,46.38,...,56.06,50.77,661.90,28.59,23.24,30.90,53.83,18.35,30.49,44.03
9,ENSG00000001167.14,"ENST00000341376.10,ENST00000353205.5",3.44,5.47,5.25,16.63,7.92,8.57,19.71,14.22,...,17.46,14.67,12.25,22.19,20.15,24.29,26.21,27.42,20.88,14.45


In [ ]:
prev = set(prev.columns.values.tolist())
new1 = set(counts_genes.columns.values.tolist())
new2 = set(tpm_genes.columns.values.tolist())
new3 = set(tpm_transcripts.columns.values.tolist())
print(len(prev), len(prev & new2), len(new1), len(new1 & new2 & new3), new2-prev, prev-new2)

In [49]:
metadata = tc.get(name='internal-19q2-9504', version=24, file='sample_info')

[##################]100% |  13.0 MiB/s | 245.8 KiB / 245.8 KiB | Time:  0:00:00


In [58]:
metadata

,DepMap_ID,stripped_cell_line_name,CCLE_name,alias,COSMIC_ID,Sanger_ID,disease,disease_sutype,disease_sub_subtype,gender,source,Achilles_n_replicates,cell_line_NNMD,culture_type,culture_medium,cas9_activity
0,ACH-000001,NIHOVCAR3,NIHOVCAR3_OVARY,OVCAR3,905933.0,2201,ovary,ovary_adenocarcinoma,ovary_high_grade_serous,Female,ATCC,NaN,NaN,NaN,NaN,NaN
1,ACH-000002,HL60,HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,905938.0,55,leukemia,AML,,Female,ATCC,NaN,NaN,NaN,NaN,NaN
2,ACH-000003,CACO2,CACO2_LARGE_INTESTINE,"CACO2, CaCo-2",NaN,,colorectal,,,Male,NaN,NaN,NaN,NaN,NaN,NaN
3,ACH-000004,HEL,HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,907053.0,783,leukemia,AML,,Male,DSMZ,2.0,-3.097997,Suspension,RPMI + 10% FBS,47.6
4,ACH-000005,HEL9217,HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,NaN,,leukemia,AML,,Male,ATCC,2.0,-2.431055,Semi-adherent,RPMI + 10% FBS,13.4
5,ACH-000006,MONOMAC6,MONOMAC6_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,NaN,908148.0,2167,leukemia,AML,,Male,DSMZ,NaN,NaN,NaN,NaN,NaN
6,ACH-000007,LS513,LS513_LARGE_INTESTINE,NaN,907795.0,569,colorectal,,,Male,ATCC,2.0,-2.586708,Adherent,RPMI + 10% FBS,52.3
7,ACH-000008,A101D,A101D_SKIN,NaN,910921.0,1806,,,,Male,ATCC,NaN,NaN,NaN,NaN,NaN
8,ACH-000009,C2BBE1,C2BBE1_LARGE_INTESTINE,NaN,910700.0,2104,colorectal,,,Male,ATCC,3.0,-4.140110,Adherent,DMEM + 10% FBS + Glutamine + Insulin-Transferr...,5.8
9,ACH-000010,NCIH2077,NCIH2077_LUNG,NCI-H1581,NaN,,lung,lung_NSC,lung_adenocarcinoma,Male,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
trainame = [val for val in new1&prev if val[:3] == 'ACH']
testname = [val for val in new1-prev if val[:3] == 'ACH']
genetolookfor = 2000
gene_var = counts_genes[trainame].var(1).values
print(len(gene_var))
sorting = np.argsort(gene_var)[-genetolookfor:]

58813


In [51]:
unregistered = set(testname) - set(metadata["DepMap_ID"].values.tolist())
unregistered

set()

In [92]:
len(counts_genes['ACH-001767']) - np.count_nonzero(counts_genes['ACH-001767'].values)

43950

In [68]:
# creating and reodering train and test sets
traindata = counts_genes[set(trainame)-unregistered].values[sorting].T
trainlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(trainame)-unregistered].columns.tolist() if val not in unregistered]

testdata = counts_genes[set(testname)-unregistered].values[sorting].T
testlabels = [metadata[metadata["DepMap_ID"]==val]["disease"].values[0] for val in counts_genes[set(testname)-unregistered].columns.tolist() if val not in unregistered]

In [61]:
traindata.shape

array([[4.4341e+03, 4.6400e+03, 5.9210e+03, ..., 4.8150e+03, 2.3967e+06,
        2.5996e+06],
       [0.0000e+00, 3.0310e+03, 6.7350e+03, ..., 3.1210e+04, 1.9364e+05,
        2.2775e+05],
       [3.9182e+03, 8.6580e+03, 9.5710e+03, ..., 3.4614e+06, 7.6804e+05,
        3.6592e+06],
       ...,
       [1.7400e+01, 3.8710e+03, 1.1838e+04, ..., 4.0900e+02, 2.5226e+05,
        5.7418e+05],
       [6.0000e+00, 1.5882e+04, 1.1291e+04, ..., 2.0010e+03, 2.8874e+05,
        6.2399e+05],
       [6.0700e+00, 8.3970e+03, 9.9610e+03, ..., 1.2568e+05, 2.7778e+05,
        3.7050e+05]])

In [69]:
# learn KNN classifier to the metadata diseases
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(traindata, trainlabels) 
predicted = neigh.predict(testdata)

In [76]:
predicted[1]

'lymphoma'

In [79]:
# get the points that failed on the classifier
failed = []
names =  counts_genes[testname].columns.tolist()
for i, pred in enumerate(predicted):
    if pred != testlabels[i]:
        failed.append(names[i])
        print(names[i],pred,testlabels[i])

ACH-001767 rhabdomyosarcoma fibroblast


In [ ]:
# plot them with TSNE, highlight the points that failed and show colors for diseases
source = ColumnDataSource(
    data=dict(
        x=gene_cn_red[0],
        y=gene_cn_red[1],
        label= gene_cn.index.tolist()
        color
    )
)
TOOLS="crosshair,pan,wheel_zoom,box_zoom,reset,hover,previewsave"
p = figure(title="gene cn", tools=TOOLS)
p.circle('x', 'y', color="#2222aa", line_width=2, source=source)
hover =p.select(dict(type=HoverTool))
hover.tooltips = OrderedDict([
    ("label", "@label"),
])

show(p)

## Save files for taiga

In [118]:
tc.update_dataset(dataset_permaname="depmap-expression-87f8",
                 upload_file_path_dict={'temp/expression.'+release+'.transcripts.tsv': 'TableTSV',
                                       'temp/expression.'+release+'.tpm.tsv': 'TableTSV',
                                       'temp/expression.'+release+'.counts.tsv': 'TableTSV'},
                 dataset_description="Updating to "+release,
                    
                 force_remove=True)

Uploading expression.19Q3interim.transcripts...
hitting https://cds.team/taiga/api/datafile/40558c8bfc414791842d94deb5c37008
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: expression.19Q3interim.transcripts properly converted and uploaded
Uploading expression.19Q3interim.tpm...
hitting https://cds.team/taiga/api/datafile/40558c8bfc414791842d94deb5c37008
Conversion and upload...:
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: expression.19Q3interim.tpm properly converted and uploaded
Uploading expression.19Q3interim.counts...
hitting https://cds.team/taiga/api/datafile/40558c8bfc414791842d94deb5c37008
Conversion and upload...:


	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3
	 Uploading to S3

	 Done: expression.19Q3interim.counts properly converted and uploaded
Creating the new version with these files:
	NEW: expression.19Q3interim.transcripts - TableTSV
	NEW: expression.19Q3interim.tpm - TableTSV
	NEW: expression.19Q3interim.counts - TableTSV
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 6d8f8e51c1394ed8ae0c1128acb85326 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/6d8f8e51c1394ed8ae0c1128acb85326


'6d8f8e51c1394ed8ae0c1128acb85326'

## Fusion post processing

In [ ]:
terra.wait_for_submission(submission_id2)

In [59]:
aggregate = refwm.get_config('Aggregate_Fusion_Calls')
aggregate

{'deleted': False,
 'inputs': {'aggregate_set_files_workflow.aggregate_set_files.disk_space': '250',
  'aggregate_set_files_workflow.aggregate_set_files.output_file_name': '"Fusions.aggregated.tsv"',
  'aggregate_set_files_workflow.aggregate_set_files.aggregate_files_script': 'workspace.aggregate_fusions_script',
  'aggregate_set_files_workflow.aggregate_set_files.num_preempt': '5',
  'aggregate_set_files_workflow.aggregate_set_files.sample_files': 'this.samples.fusion_predictions_abridged',
  'aggregate_set_files_workflow.aggregate_set_files.memory': '3'},
 'methodConfigVersion': 7,
 'methodRepoMethod': {'methodName': 'Aggregate_files_set',
  'methodVersion': 2,
  'methodNamespace': 'gkugener',
  'methodUri': 'agora://gkugener/Aggregate_files_set/2',
  'sourceRepo': 'agora'},
 'name': 'Aggregate_Fusion_Calls',
 'namespace': 'gkugener',
 'outputs': {'aggregate_set_files_workflow.aggregate_set_files.output_merged_file': 'this.fusions_star'},
 'prerequisites': {},
 'rootEntityType': 'sam

In [60]:
refwm.update_config(aggregate)
submission_id2 = refwm.create_submission(aggregate['name'], 'All_samples')

Successfully updated configuration gkugener/Aggregate_Fusion_Calls
Successfully created submission c853b79b-c226-4b29-a718-5875b7f74e10.


In [ ]:
terra.wait_for_submission(submission_id2)

In [10]:
refwm.get_sample_sets().loc['All_samples']['fusions_star']

fusions_star                                                                     NaN
mergedExonUsageRObj_hg38                                                         NaN
rsem_genes_expected_count                                                        NaN
rsem_genes_tpm                                                                   NaN
rsem_transcripts_expected_count                                                  NaN
rsem_transcripts_isopct                                                          NaN
rsem_transcripts_tpm                                                             NaN
samples                            [dm_ACH-000001, dm_ACH-000002, dm_ACH-000003, ...
Name: all_samples, dtype: object

In [37]:
! gsutil cp $aggregated "temp/expression.fusion.tsv"

Copying gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/15ab70c1-73af-44bc-9aeb-750ef6d70bc8/aggregate_set_files_workflow/7d6bdfe1-27fb-478b-ade1-fcda622c4960/call-aggregate_set_files/Fusions.aggregated.tsv...
/ [1 files][ 30.2 KiB/ 30.2 KiB]                                                
Operation completed over 1 objects/30.2 KiB.                                     


### Overview

This document contains the code used to generate the unfiltered and filtered versions of the fusion datasets for the release. The bottom of the document also contains some comparisons between the release fusion dataset, CCLE2 fusion calls, and the translocation data from CCLE2.

In [ ]:
%%R
source('../JKBio/gkugener/RScripts/load_libraries_and_annotations.R')
source("CCLE_postp_function.R")
filepath <- 'temp/expression.fusion.tsv'
release <- '19Q3interim'

## Generate filtered fusion table

Release: `r release`

We want to apply filters to the fusion table to reduce the number of artifacts in the dataset. Specifically, we filter the following:

* Remove fusions involving mitochondrial chromosomes, or HLA genes, or immunoglobulin genes
* Remove red herring fusions (from STAR-Fusion annotations column)
* Remove recurrent in CCLE (>= 25 samples)
* Remove fusion with (SpliceType=" INCL_NON_REF_SPLICE" and LargeAnchorSupport="No" and FFPM < 0.1)
* Remove fusions with FFPM < 0.05 (STAR-Fusion suggests using 0.1, but looking at the translocation data, this looks like it might be too aggressive)

In [15]:
%%R
unfiltered_fusions <- read_fusions(filepath)
filtered_fusions <- filter_fusions(unfiltered_fusions)

In [18]:
%%R
# Save the files (to be uploaded to taiga)
write.table(
  unfiltered_fusions,
  file = paste0('temp/fusions.',release, '.unfiltered.tsv'),
  sep = '\t', quote = F, row.names = F
)
write.table(
  filtered_fusions,
  file = paste0('temp/fusions.', release, '.filtered.tsv'),
  sep = '\t', quote = F, row.names = F
)

# Validation

## Validation Protocol:

to validate fusions, one should be able to list all cells with known fusions (i.e. elwing sarcoma) and check for each new cell in this set of knownfusioncells, if the fusion is present or not. and validate the fusion quality this way.

In [81]:
# check that all cells lines are present on fusion unfiltered
df = pd.read_csv('temp/fusions.'+release+'.unfiltered.tsv', sep='\t')
new = set([i.split('_')[1][:10] for i in list(set(df["DepMap_ID"].tolist()))])
print(prev - new, new-prev)

{'gene_id', 'transcript_id(s)'} {'ACH-001616'}


In [80]:
# remove fusions from the same samples as for that failed expression threshold


{'gene_id', 'transcript_id(s)'}

# Uploading to Taiga

In [19]:
tc.update_dataset(dataset_permaname="depmap-fusions-7990",
                     upload_file_path_dict={'temp/fusions.'+release+'.filtered.tsv': 'TableTSV',
                                        'temp/fusions.'+release+'.unfiltered.tsv': 'TableTSV'},
                 dataset_description="Updating to "+release,
                 force_remove=True)


Uploading fusions.19Q3interim.filtered...
hitting https://cds.team/taiga/api/datafile/14863a8199a74255b73ded5c6375ad0e
Conversion and upload...:

	 Done: fusions.19Q3interim.filtered properly converted and uploaded
Uploading fusions.19Q3interim.unfiltered...
hitting https://cds.team/taiga/api/datafile/14863a8199a74255b73ded5c6375ad0e
Conversion and upload...:
	 Uploading to S3

	 Done: fusions.19Q3interim.unfiltered properly converted and uploaded
Creating the new version with these files:
	NEW: fusions.19Q3interim.filtered - TableTSV
	NEW: fusions.19Q3interim.unfiltered - TableTSV
hitting https://cds.team/taiga/api/datasetVersion

Dataset version with id 113b84b79aa2481fa909a68bf4093251 created. You can access to this dataset version directly with this url: https://cds.team/taiga/dataset_version/113b84b79aa2481fa909a68bf4093251


'113b84b79aa2481fa909a68bf4093251'

# IF want to merge here instead of on Terra:

In [7]:
samples = ['ibm_ACH-001616','dm_ACH-000309', 'ibm_ACH-001852']
res = []
terrasamp = refwm.get_samples()
for i, sample in enumerate(samples):
    res.append(terrasamp.loc[sample])
    genes_fusion = res[i]['fusion_predictions_abridged']
    rsem_genes_transcripts = res[i]['rsem_isoforms']
    rsem_genes_expected_count = res[i]['rsem_genes']
    ! gsutil cp $rsem_genes_expected_count 'temp/' && gsutil cp $rsem_genes_transcripts 'temp/' && gsutil cp $genes_fusion 'temp/'

In [86]:
mainres = refwm.get_sample_sets().loc['DM19Q2_PATHS_CORRECTED_V2']
maingenes_fusion = mainres['fusions_star']
mainrsem_genes_tpm = mainres['rsem_genes_tpm']
mainrsem_genes_transcripts = mainres['rsem_transcripts_tpm']
mainrsem_genes_expected_count = mainres['rsem_genes_expected_count']
! gsutil cp $mainrsem_genes_expected_count "temp/" && gsutil cp $mainrsem_genes_transcripts "temp/" && gsutil cp $maingenes_fusion "temp/expression.fusion.tsv" && gsutil cp $mainrsem_genes_tpm "temp/"

Copying gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/3a120a00-cc0e-4e61-81f0-2efb6f39a40f/rsem_aggregate_results_workflow/7bb43c84-bedb-402a-bdfa-c1e20cba11e5/call-rsem_aggregate_results/DM19Q2_PATHS_CORRECTED_V2.rsem_genes_expected_count.txt.gz...
\ [1 files][ 68.2 MiB/ 68.2 MiB]                                                
Operation completed over 1 objects/68.2 MiB.                                     
Copying gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/3a120a00-cc0e-4e61-81f0-2efb6f39a40f/rsem_aggregate_results_workflow/7bb43c84-bedb-402a-bdfa-c1e20cba11e5/call-rsem_aggregate_results/DM19Q2_PATHS_CORRECTED_V2.rsem_transcripts_tpm.txt.gz...
/ [1 files][228.9 MiB/228.9 MiB]                                                
Operation completed over 1 objects/228.9 MiB.                                    
Copying gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/7ab5a444-1b66-41ea-b1fb-053bc92158ef/aggregate_set_files_workflow/48cec2fa-c2cf-4d56-83f1-d0fe68c3d66d/call

In [ ]:
addSamplesRSEMToMain(res,mainres)

In [19]:
genes_fusion = ['temp/'+val['fusion_predictions_abridged'].split('/')[-1] for val in res]
addToMainFusion(genes_fusion,'temp/expression.fusion.tsv')

> /home/jeremie/ccle_processing/CCLE_postp_function.py(135)addToMainFusion()
-> maindata = pd.read_csv(main_filename, sep='\t')
(Pdb) n
> /home/jeremie/ccle_processing/CCLE_postp_function.py(136)addToMainFusion()
-> if '.' in maindata["DepMap_ID"][0]:
(Pdb) n
> /home/jeremie/ccle_processing/CCLE_postp_function.py(137)addToMainFusion()
-> maindata["DepMap_ID"] = [i[0] for i in maindata["DepMap_ID"].str.split('.').tolist()]
(Pdb) n
> /home/jeremie/ccle_processing/CCLE_postp_function.py(138)addToMainFusion()
-> samples = set(maindata["DepMap_ID"].tolist())
(Pdb) n
> /home/jeremie/ccle_processing/CCLE_postp_function.py(139)addToMainFusion()
-> with open(main_filename, 'a') as f:
(Pdb) n
> /home/jeremie/ccle_processing/CCLE_postp_function.py(140)addToMainFusion()
-> for input_filename in input_filenames:
(Pdb) n
> /home/jeremie/ccle_processing/CCLE_postp_function.py(141)addToMainFusion()
-> df = pd.read_csv(input_filename,  sep='\t')
(Pdb) n
> /home/jeremie/ccle_processing/CCLE_postp_functi

BdbQuit: 